# <B> SageMaker pileline with `MLflow` for Anormaly Detection based on AutoEncoder </B>
* Container: codna_pytorch_p310
* [Example codes](https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-mlflow)

## [중요] Pipeline에 사용되는 주요 사용자 파라미터는 config.ini 파일(./pipeline_config/config.ini)로 관리되고 있습니다. 
 - 특히!! mlflow arn도 config.ini 에 넣어 주어야 합니다!!

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 1. Pipeline definition

##### 1. "./pipeline_config/config.ini" 파일열기
##### 2. "tracking_server_arn"을 자신의 mlflow arn으로 반드시 수정할 것 !!!

In [2]:
import os
import time
import boto3
import mlflow
import argparse
from pprint import pprint
from pipeline_config.config import config_handler

from sagemaker.pytorch.estimator import PyTorch
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.steps import CacheConfig, ProcessingStep, TrainingStep
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput, FrameworkProcessor
from sagemaker.workflow.retry import StepRetryPolicy, StepExceptionTypeEnum, SageMakerJobExceptionTypeEnum, SageMakerJobStepRetryPolicy

from mlflow.tracking import MlflowClient

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/18/25 16:00:20] INFO     Found credentials from IAM Role:                                   ]8;id=856325;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=839674;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
class mlops_pipeline():

    def __init__(self, args):

        self.args = args

        self.strRegionName = self.args.config.get_value("COMMON", "region")
        self._env_setting()

    def _env_setting(self, ):

        self.strPrefix = self.args.config.get_value("COMMON", "prefix")
        self.strExecutionRole = self.args.config.get_value("COMMON", "role")
        self.strBucketName = self.args.config.get_value("COMMON", "bucket")
        self.strModelName = self.args.config.get_value("COMMON", "model_name")
        self.strPipelineName = "-".join([self.strPrefix, self.strModelName])
        self.tracking_server_arn = self.args.config.get_value("COMMON", "tracking_server_arn")
        self.mlflow_exp_name = self.args.config.get_value("COMMON", "mlflow_exp_name")
        self.mlflow_run_name = ExecutionVariables.PIPELINE_EXECUTION_ID
        
        ## mlflow
        client = MlflowClient()
        mlflow.set_tracking_uri(self.tracking_server_arn)
        
        try:
            # experiment 존재 여부 확인
            experiment = client.get_experiment_by_name(self.mlflow_exp_name)

            if experiment is None:
                # experiment가 없으면 새로 생성
                experiment_id = mlflow.create_experiment(self.mlflow_exp_name)
                print(f"Created new experiment '{self.mlflow_exp_name}' with ID: {experiment_id}")
            else:
                experiment_id = experiment.experiment_id
                print(f"Using existing experiment '{self.mlflow_exp_name}' with ID: {experiment_id}")

            # experiment 설정
            mlflow.set_experiment(self.mlflow_exp_name)

        except Exception as e:
            print(f"Error in run_under_experiment: {str(e)}")
        
        session = boto3.Session()
        self.credentials = session.get_credentials()

        self.cache_config = CacheConfig(
            enable_caching=self.args.config.get_value("PIPELINE", "enable_caching", dtype="boolean"),
            expire_after=self.args.config.get_value("PIPELINE", "expire_after")
        )

        self.retry_policies=[
            # retry when resource limit quota gets exceeded
            SageMakerJobStepRetryPolicy(
                exception_types=[SageMakerJobExceptionTypeEnum.RESOURCE_LIMIT],
                expire_after_mins=180,
                interval_seconds=60,
                backoff_rate=1.0
            ),
        ]

        if self.args.config.get_value("LOCALMODE", "mode", dtype="boolean"): self.pipeline_session = LocalPipelineSession()
        else: self.pipeline_session = PipelineSession()

        print (f" == Envrionment parameters == ")
        print (f"   SAGEMAKER-ROLE-ARN: {self.strExecutionRole}")
        print (f"   PREFIX: {self.strPrefix}")
        print (f"   BUCKET: {self.strBucketName}")

    def _step_preprocessing(self, ):
        
        if self.args.config.get_value("LOCALMODE", "mode", dtype="boolean"): instance_type = "local"
        else: instance_type = self.args.config.get_value("PREPROCESSING", "instance_type")
        
        strPrefixPrep = "/opt/ml/processing/"
        strDataPath = self.args.config.get_value("PREPROCESSING", "data_path")
        strTrainDataName = self.args.config.get_value("PREPROCESSING", "data_name")

        # network_config로 받으면 된다
        prep_processor = FrameworkProcessor(
            estimator_cls=PyTorch,
            framework_version=self.args.config.get_value("PREPROCESSING", "framework_version"),
            py_version="py310",
            image_uri=None,
            instance_type=instance_type,
            instance_count=self.args.config.get_value("PREPROCESSING", "instance_count", dtype="int"),
            role=self.strExecutionRole,
            base_job_name="preprocessing", # bucket에 보이는 이름 (pipeline으로 묶으면 pipeline에서 정의한 이름으로 bucket에 보임)
            sagemaker_session=self.pipeline_session,
            #env={"MLFLOW_TRACKING_ARN": self.tracking_server_arn}
        )

        step_args = prep_processor.run(
            #job_name="preprocessing", ## 이걸 넣어야 캐시가 작동함, 안그러면 프로세서의 base_job_name 이름뒤에 날짜 시간이 붙어서 캐시 동작 안함
            #git_config=git_config,
            code='preprocessing-mlflow.py', #소스 디렉토리 안에서 파일 path
            source_dir= "./src/preprocessing", #현재 파일에서 소스 디렉토리 상대경로 # add processing.py and requirements.txt here
            inputs=[
                ProcessingInput(
                    input_name="input-data",
                    source=strDataPath,
                    destination=os.path.join(strPrefixPrep, "input")
                ),
            ],
            outputs=[
                ProcessingOutput(
                    output_name="output-data",
                    source=os.path.join(strPrefixPrep, "output"),
                    destination=os.path.join(
                        "s3://{}".format(self.strBucketName),
                        self.strPipelineName,
                        "preprocessing",
                        "output"
                    )
                ),
            ],
            arguments=[
                "--proc_prefix", strPrefixPrep, \
                "--shingle_size", str(self.args.config.get_value("PREPROCESSING", "shingle_size", dtype="int")), \
                "--train_data_name", strTrainDataName,
                "--mlflow_tracking_arn", self.tracking_server_arn,
                "--experiment_name", self.mlflow_exp_name,
                "--mlflow_run_name", self.mlflow_run_name
            ]
        )

        self.preprocessing_process = ProcessingStep(
            name="PreprocessingProcess", ## Processing job이름
            step_args=step_args,
            cache_config=self.cache_config,
        )
        
        print ("  \n== Preprocessing Step ==")
        print ("   \nArgs: ")

        
        for key, value in self.preprocessing_process.arguments.items():
            print ("===========================")
            print (f'key: {key}')
            pprint (value)
            
        print (type(self.preprocessing_process.properties))
            

    def _step_training(self, ):
        
        if self.args.config.get_value("LOCALMODE", "mode", dtype="boolean"):
            instance_type = "local_gpu"
            environment = {
                "AWS_ACCESS_KEY_ID": self.credentials.access_key,
                "AWS_SECRET_ACCESS_KEY": self.credentials.secret_key,
                "AWS_SESSION_TOKEN": self.credentials.token,
                "AWS_REGION": self.strRegionName
            }
        else:
            instance_type = self.args.config.get_value("TRAINING", "instance_type")
            environment={
                "MLFLOW_TRACKING_ARN": self.tracking_server_arn,
                "EXPERIMENT_NAME": self.mlflow_exp_name,
                "MLFLOW_RUN_NAME": self.mlflow_run_name
            }

        dicHyperParams = {
            "epochs":"150",
            "batch_size":"128", 
            "lr":"1e-2",
            "shingle_size":str(self.args.config.get_value("PREPROCESSING", "shingle_size", dtype="int")),
            "num_features":"4",
            "emb_size":"4",
            "workers":"2"
        }

        strOutputPath = os.path.join(
            "s3://{}".format(self.strBucketName),
            self.strPipelineName,
            "training",
            "model-output"
        )

        strCodeLocation = os.path.join(
            "s3://{}".format(self.strBucketName),
            self.strPipelineName,
            "training",
            "backup_codes"
        )

        num_re = "([0-9\\.]+)(e-?[[01][0-9])?"
        metric_definitions = [
            {"Name": "Train loss", "Regex": f"loss={num_re}"},
            {"Name": "Train cos", "Regex": f"wer:{num_re}"},
            {"Name": "Val cos", "Regex": f"wer:{num_re}"}
        ]

        bSpotTraining = False
        if bSpotTraining:
            nMaxWait = 1*60*60
            nMaxRun = 1*60*60

        else:
            nMaxWait = None
            nMaxRun = 1*60*60

        bUseTrainWarmPool = True ## training image 다운받지 않음, 속도 빨라진다
        if bUseTrainWarmPool: nKeepAliveSeconds = 3600 ## 최대 1시간 동안!!, service quota에서 warmpool을 위한 request 필요
        else: nKeepAliveSeconds = None
        if bSpotTraining:
            bUseTrainWarmPool = False # warmpool은 spot instance 사용시 활용 할 수 없음
            nKeepAliveSeconds = None
        
        self.estimator = PyTorch(
            entry_point="main-mlflow.py", # the script we want to run
            source_dir="./src/training", # where our conf/script is
            #git_config=git_config,
            role=self.strExecutionRole,
            instance_type=instance_type,
            instance_count=self.args.config.get_value("TRAINING", "instance_count", dtype="int"),
            image_uri=None,
            framework_version=self.args.config.get_value("TRAINING", "framework_version"),
            py_version="py310",
            volume_size=128,
            code_location=strCodeLocation,
            output_path=strOutputPath,
            disable_profiler=True,
            debugger_hook_config=False,
            hyperparameters=dicHyperParams, #{'config-path': 'conf'},
            #distribution={"smdistributed":{"dataparallel":{"enabled":True, "fp16": True}}},
            sagemaker_session=self.pipeline_session,
            metric_definitions=metric_definitions,
            max_run=nMaxRun,
            use_spot_instances=bSpotTraining,  # spot instance 활용
            max_wait=nMaxWait,
            keep_alive_period_in_seconds=nKeepAliveSeconds,
            enable_sagemaker_metrics=True,
            environment=environment
        )

        step_training_args = self.estimator.fit(
            job_name="training",
            inputs={
                "train": self.preprocessing_process.properties.ProcessingOutputConfig.Outputs["output-data"].S3Output.S3Uri,
                "validation": self.preprocessing_process.properties.ProcessingOutputConfig.Outputs["output-data"].S3Output.S3Uri,
            },
            logs="All",
        )
        
        self.training_process = TrainingStep(
            name="TrainingProcess",
            step_args=step_training_args,
            cache_config=self.cache_config,
            #depends_on=[self.preprocessing_process],
            retry_policies=self.retry_policies
        )

        print ("  \n== Training Step ==")
        print ("   \nArgs: ")

        for key, value in self.training_process.arguments.items():
            print ("===========================")
            print (f'key: {key}')
            pprint (value)
            
    def _step_model_registration(self, ):
        
        self.strModelPackageGroupName = "-".join(["MPG", self.strPrefix, self.strModelName])

        model = PyTorchModel(
            source_dir="./src/deploy",
            entry_point="inference.py",
            model_data=self.training_process.properties.ModelArtifacts.S3ModelArtifacts,
            role=self.strExecutionRole,
            framework_version=self.args.config.get_value("MODEL_REGISTER", "framework_version"),
            py_version='py310',
            model_server_workers=1,
            code_location=os.path.join(
                "s3://",
                self.strBucketName,
                self.strPipelineName,
                "inference",
                "model"
            ),
            sagemaker_session=self.pipeline_session,
        )

        step_args = model.register(
            content_types=["application/json", "file-path/raw-bytes", "text/csv"],
            response_types=["application/json"],
            inference_instances=self.args.config.get_value("MODEL_REGISTER", "inference_instances", dtype="list"),
            transform_instances=self.args.config.get_value("MODEL_REGISTER", "transform_instances", dtype="list"),
            model_package_group_name=self.strModelPackageGroupName,
            approval_status=self.args.config.get_value("MODEL_REGISTER", "model_approval_status_default"),
            ## “Approved”, “Rejected”, or “PendingManualApproval” (default: “PendingManualApproval”).
        )

        self.register_process = ModelStep(
            name="ModelRegisterProcess",
            step_args=step_args,
            #depends_on=[self.evaluation_process]
        )

    def _step_deploy(self, ):
        
        strInstanceType = self.args.config.get_value("DEPLOY", "processing_instance_type")
        nInstanceCount = self.args.config.get_value("DEPLOY", "processing_instance_count", dtype="int")
        strDepolyInstanceType = self.args.config.get_value("DEPLOY", "instance_type")
        strEndpointName = f'endpoint--{self.strPipelineName}-{int(time.time())}'
        strProcPrefixPath = "/opt/ml/processing"
        
        deploy_processor = FrameworkProcessor(
            estimator_cls=PyTorch,
            framework_version=self.args.config.get_value("DEPLOY", "processing_framework_version"),
            py_version="py310",
            image_uri=None,
            role=self.strExecutionRole,
            instance_type=strInstanceType,
            instance_count=nInstanceCount,
            base_job_name="deploy", # bucket에 보이는 이름 (pipeline으로 묶으면 pipeline에서 정의한 이름으로 bucket에 보임)
            sagemaker_session=self.pipeline_session
        )
        
        step_deploy_args = deploy_processor.run(
            code="deploy.py",
            source_dir="src/deploy/",
            arguments=[
                "--prefix_deploy", strProcPrefixPath, \
                "--region", self.strRegionName, \
                "--instance_type", strInstanceType, \
                "--depoly_instance_type", strDepolyInstanceType, \
                "--model_package_group_name", self.strModelPackageGroupName, \
                "--endpoint_name", strEndpointName, \
                "--execution_role", self.strExecutionRole, \
            ],
            job_name="deploy",
        )
        
        
        self.deploy_process = ProcessingStep(
            name="DeployProcess", ## Processing job이름
            step_args=step_deploy_args,
            depends_on=[self.register_process],
            cache_config=self.cache_config,
            retry_policies=self.retry_policies
        )
        
        print ("  \n== Deploy Step ==")
        print ("   \nArgs: ")

        for key, value in self.deploy_process.arguments.items():
            print ("===========================")
            print (f'key: {key}')
            pprint (value)
            
    def _get_pipeline(self, ):

        pipeline = Pipeline(
            name=self.strPipelineName,
            steps=[self.preprocessing_process, self.training_process, self.register_process, self.deploy_process],
            #steps=[self.preprocessing_process, self.training_process],
            sagemaker_session=self.pipeline_session
        )

        return pipeline

    def execution(self, ):

        self._step_preprocessing()
        self._step_training()
        self._step_model_registration()
        self._step_deploy()

        pipeline = self._get_pipeline()
        pipeline.upsert(role_arn=self.strExecutionRole) ## Submit the pipeline definition to the SageMaker Pipelines service 
        execution = pipeline.start()
        desc = execution.describe()

        print ("PipelineArn: ", desc["PipelineArn"])
        print (execution.describe())

In [4]:
parser = argparse.ArgumentParser()
args, _ = parser.parse_known_args()
args.config = config_handler()

print("Received arguments {}".format(args))
os.environ['AWS_DEFAULT_REGION'] = args.config.get_value("COMMON", "region")

pipe_tr = mlops_pipeline(args)
pipe_tr.execution()

====== config info. ======
  LOCALMODE: mode:False
  COMMON: prefix:KC-SM-PIPELINE
  COMMON: region:us-east-1
  COMMON: role:arn:aws:iam::205457763032:role/service-role/AmazonSageMakerExecutionRole-sagemaker-stack
  COMMON: bucket:sm-anomaly-detection-kichul-04101
  COMMON: model_name:MODEL-1
  COMMON: image_uri_prep:419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-prep:latest
  COMMON: image_uri:419974056037.dkr.ecr.us-east-1.amazonaws.com/mlops-image-tr:latest
  COMMON: tracking_server_arn:arn:aws:sagemaker:us-east-1:205457763032:mlflow-tracking-server/mlflow-d-mbjfccgc4fn5
  COMMON: mlflow_exp_name:anomaly-detection-exp-trial-1
  PIPELINE: enable_caching:True
  PIPELINE: expire_after:T48H
  PREPROCESSING: data_path:s3://sm-anomaly-detection-kichul-04101/data
  PREPROCESSING: data_name:merged_clicks_1T.csv
  PREPROCESSING: framework_version:2.0.0
  PREPROCESSING: instance_type:ml.m5.xlarge
  PREPROCESSING: instance_count:1
  PREPROCESSING: shingle_size:4
  TRAINING: framework

[02/18/25 16:00:22] INFO     Found credentials from IAM Role:                                   ]8;id=876502;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=126977;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

2025/02/18 16:00:22 INFO mlflow.tracking.fluent: Experiment with name 'anomaly-detection-exp-trial-1' does not exist. Creating a new experiment.


                    INFO     Found credentials from IAM Role:                                   ]8;id=889278;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=224307;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[02/18/25 16:00:23] INFO     Found credentials from IAM Role:                                   ]8;id=772183;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=684531;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

...

                    WARNING  Popping out 'ModelPackageName' from the pipeline definition by        ]8;id=595390;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=159382;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

[02/18/25 16:00:27] INFO     Uploaded src/deploy/ to                                             ]8;id=766821;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=46349;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/processing.py#1961\1961]8;;\
                             s3://sagemaker-us-east-1-205457763032/KC-SM-PIPELINE-MODEL-1/code/4                   
                             ff350db9b90ed0a4078047abe55376f2b0ef8c8ba06b7cf7508cff14b8dbf39/sou                   
                             rcedir.tar.gz                                                                         

                    INFO     runproc.sh uploaded to                                              ]8;id=664025;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=658623;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/processing.py#2055\2055]8;;\
                             s3://sagemaker-us-east-1-205457763032/KC-SM-PIPELINE-MODEL-1/code/8                   
                             5bc289a20f19a41a804712262cdbfea0dc439d84425deaf4f38f05d578fe701/run                   
                             proc.sh                                                                               

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=497102;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=95463;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

PipelineArn:  arn:aws:sagemaker:us-east-1:205457763032:pipeline/KC-SM-PIPELINE-MODEL-1
{'PipelineArn': 'arn:aws:sagemaker:us-east-1:205457763032:pipeline/KC-SM-PIPELINE-MODEL-1', 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:205457763032:pipeline/KC-SM-PIPELINE-MODEL-1/execution/510fzwwdamp8', 'PipelineExecutionDisplayName': 'execution-1739894427580', 'PipelineExecutionStatus': 'Executing', 'CreationTime': datetime.datetime(2025, 2, 18, 16, 0, 27, 522000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 2, 18, 16, 0, 27, 522000, tzinfo=tzlocal()), 'CreatedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::205457763032:assumed-role/AmazonSageMakerExecutionRole-sagemaker-stack/SageMaker', 'PrincipalId': 'AROAS7VR4VLMC6WQCYJ22:SageMaker'}}, 'LastModifiedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::205457763032:assumed-role/AmazonSageMakerExecutionRole-sagemaker-stack/SageMaker', 'PrincipalId': 'AROAS7VR4VLMC6WQCYJ22:SageMaker'}}, 'ResponseMetadata': {'RequestId': 'ebd0e85f-0198

In [5]:
client = MlflowClient()

In [6]:
experiments = client.search_experiments()

                    INFO     Found credentials from IAM Role:                                   ]8;id=341010;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=420944;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [7]:
experiments[0].experiment_id

'2'

In [8]:
mlflow.set_experiment(experiments[0].name)

[02/18/25 16:00:28] INFO     Found credentials from IAM Role:                                   ]8;id=556740;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=817014;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

<Experiment: artifact_location='s3://sagemaker-studio-205457763032-fd9aa650/mlflow/d-mbjfccgc4fn5/2', creation_time=1739894422993, experiment_id='2', last_update_time=1739894422993, lifecycle_stage='active', name='anomaly-detection-exp-trial-1', tags={}>